In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
import csv
import statistics
from scipy import stats

In [ ]:
def create_table(inputfile,outputfile):
    '''Code to create results table that shows the average number of responses worker gave to prompt
       Arg1 is the input file that is generated by get_results.py
       Arg2 is the desired output files that is a csv formatted from the dictionary created below'''
    dict_keys = ['WorkerID','AssignmentID','Prompt1.0','Prompt2.0','Prompt3.0','Prompt4.0','Prompt5.0','Prompt6.0','Prompt7.0',
                'Prompt8.0','Prompt9.0','Prompt10.0','Median','Size']   # initializes all of the keys we will need 
    dict_dict = {}                                                      # the dictionary we will use for data
    with open(inputfile, 'r') as input:
        reader = csv.reader(input)
        header = next(reader)
        for row in reader:
            workerID = row[1]                                           # gets the WorkerID from CSV
            assignmentID = row[0]
            responses = row[9:59]                                       # responses relevant to our analysis are in these columns
            if workerID in dict_dict.keys():                            # checks if the worker is already in our dictionary
                worker_dict = dict_dict[workerID]                       # if so we temporarily pull their subdictionary
                prev_n = worker_dict['Size']                            # we need to update the number of HITs worker completed
                worker_dict['AssignmentID'].append(assignmentID)        # we need to append this assignmentID to the ids of the HITs the worker completed
                values = []
                for i in range(0,len(responses),5):                     # Each prompt has up to 5 responses so we want to check five at a time
                    cnt = int(responses[i]!='NA') + int(responses[i+1]!='NA') + int(responses[i+2]!='NA') + int(responses[i+3]!="NA") + int(responses[i+4]!='NA')
                    updated_num = (worker_dict['Prompt{}'.format((i+5)/5)] * prev_n + cnt) / (prev_n+1)      # updates the count for the average number of responses this worker provides
                    values.append(updated_num)                                                               # add the count for this prompt to values for the purposes of getting the median of the values                                                          
                    worker_dict['Prompt{}'.format((i+5)/5)] = updated_num                                    # updates the value in the worker dictionary for this worker's avg number of responses to prompt
                worker_dict['Median'] = statistics.median(values)                                            # records the Median
                worker_dict['Size'] = prev_n+1                                                               # updates size to reflect the number of HITs the worker completed
                dict_dict[workerID] = worker_dict                                                            # puts the updated worker dictionary back into the dictionary of dictionaries
            else:
                temp_dict = {}                                          # in the case that the worker does not already exist in our dictionary of dicionaries we create a temporary dictionary for them
                temp_dict['WorkerID'] = workerID                        # next 3 lines begin to fill in basic fields
                temp_dict['AssignmentID'] = [assignmentID]
                temp_dict['Size'] = 1
                values = []
                for i in range(0,len(responses),5):                     # same loop as above except do not have to account for the values already in there
                    cnt = int(responses[i]!='NA') + int(responses[i+1]!='NA') + int(responses[i+2]!='NA') + int(responses[i+3]!="NA") + int(responses[i+4]!='NA')
                    temp_dict['Prompt{}'.format((i+5) / 5)] = cnt
                    values.append(cnt)
                temp_dict['Median'] = statistics.median(values)             
                dict_dict[workerID] = temp_dict
    
    with open(outputfile,'w') as output:                                # writes the information we have just stored to a csv file
        wr = csv.DictWriter(output,dict_keys)
        wr.writeheader()
        wr.writerows(dict_dict.values())

In [2]:
def graph_table(tablefile):
    '''Code to graph the results producedby the method create_table. Graphs results in a line graph
       Arg1 is the file to the output table produced by create_table and is used to graph results'''
    xs = np.arange(1,11)
    prompt_dict = {}
    for i in range(1,11):
        prompt_dict[i] = []
    with open(tablefile, 'r') as input:
        reader = csv.reader(input)
        header = next(reader)
        for row in reader:
            results = [float(val) for val in row[2:12]]
            for i,val in enumerate(results):
                prompt_dict[i+1].append(results[i])
            plt.scatter(xs,results)
    medians = []
    for lis in prompt_dict.values():
        medians.append(statistics.median(lis))
    slope, intercept, r_value, p_value, std_err = stats.linregress(xs, medians)
    plt.plot(xs,intercept+slope*xs,'r--')
    plt.axis([0,10,1,5])
    plt.ylabel("Number of Responses")
    plt.xlabel("Prompt Number")
    plt.show()

SyntaxError: unexpected EOF while parsing (<ipython-input-2-2794d4f6c91a>, line 1)

In [1]:
if __name__=="__main__":
    #create_table(sys.argv[1],sys.argv[2])
    graph_table(sys.argv[2])

SyntaxError: unexpected EOF while parsing (<ipython-input-1-9887ff516351>, line 1)